# 17_Defining_Many_to_Many_Relationships

## ✍️ Overview

In this section, you'll learn how to implement a Many-to-Many relationship in Spring Data JPA using annotations. We'll explain each annotation — where it is used, why it is used, and how it connects your domain models.

---

## 🧑‍🏠 `User` Entity

```java
@Entity
@Table(name = "users")
public class User {

    @Id
    @GeneratedValue(strategy = GenerationType.IDENTITY)
    @Column(name="id")
    private Long id;

    @Column(name = "name")
    private String name;

    @Column(name = "email")
    private String email;

    @Column(name = "password")
    private String password;

    @OneToMany(mappedBy = "user")
    @Builder.Default
    private List<Address> addresses = new ArrayList<>();

    // 📂 Defining the Many-to-Many Relationship
    @ManyToMany
    @JoinTable(
        name = "user_tags", // 🏛️ Join table name in DB
        joinColumns = @JoinColumn(name = "user_id"), // ← Refers to this (User) side's foreign key
        inverseJoinColumns = @JoinColumn(name = "tag_id") // → Refers to the other side's foreign key
    )
    @Builder.Default
    private Set<Tag> tags = new HashSet<>();

    public void addTag(String tagName) {
        var tag = new Tag(tagName);
        tags.add(tag);
        tag.getUsers().add(this); // Ensure both sides are updated
    }

    public void addAddress(Address address) {
        this.addresses.add(address);
        address.setUser(this);
    }

    public void removeAddress(Address address) {
        this.addresses.remove(address);
        address.setUser(null);
    }
}
```

### 🔹 Explanation of Annotations:

* `@ManyToMany`: Defines a many-to-many relationship.
* `@JoinTable`: Specifies the join table that holds foreign keys for both related entities.
* `@JoinColumn`: Defines the foreign key column for this entity.
* `@Builder.Default`: Needed to avoid null pointer exceptions when using Lombok's `@Builder`.

---

## 🌟 `Tag` Entity

```java
@Entity
@Table(name = "tags")
public class Tag {

    public Tag(String name) {
        this.name = name;
    }

    @Id
    @GeneratedValue(strategy = GenerationType.IDENTITY)
    private Long id;

    @Column(name="name")
    private String name;

    @ManyToMany(mappedBy = "tags")
    @ToString.Exclude
    private Set<User> users = new HashSet<>();
}
```

### 🔹 Explanation:

* `@ManyToMany(mappedBy = "tags")`: This tells JPA that the `User` entity owns the relationship.

  * ✉️ You only need `@JoinTable` on one side — **the owning side**, which is `User` in this case.
* `@ToString.Exclude`: Prevents circular reference when printing the object.

---

## 🚀 Usage Example (Main App)

```java
@SpringBootApplication
public class StoreApplication {

    public static void main(String[] args) {
        ApplicationContext context = SpringApplication.run(StoreApplication.class, args);

        var user = User.builder()
                .name("John")
                .password("1234")
                .email("john@gmail.com")
                .build();

        user.addTag("tag1");

        System.out.println(user);
    }
}
```

---

## 🚀 Summary

| Annotation    | Location                 | Purpose                                      |
| ------------- | ------------------------ | -------------------------------------------- |
| `@ManyToMany` | On both `User` and `Tag` | Declares the bidirectional many-to-many link |
| `@JoinTable`  | Only in `User`           | Defines join table and foreign keys          |
| `mappedBy`    | In `Tag`                 | Indicates `User` owns the relationship       |

Let me know if you want to persist this relationship to DB or use repositories and services next 🚀
